## Results Section 6.5: Approximate Circuit Synthesis
We present the results for Section 6.5. Specifically, we reproduce table 7 here.

### Imports and setup

In [ ]:
import pandas as pd 
from analyzer import main_analysis
import os

source_file = "../../data/times.csv"
source_file_optimization = "../../data/optimization.csv"
source_output = '../../data/output'
output_folder = "../../data/paper"
gheorghiu_data_path = '../../data/baselines/gheorgiu'

### Helper functions

In [ ]:
def extract_results_baseline_epsilon(file):
    """
    Extracts results from a file and returns them as a pandas DataFrame.

    Parameters:
    file (str): The path to the file containing the results.

    Returns:
    pandas.DataFrame: A DataFrame containing the extracted results with columns: "name", "epsilon", "time", "t_count".
    """

    new_lines = []
    with open(file, "r") as f:
        current_time = 0
        start_time = False
        for line in f.readlines():
            if ".txt" in line or line.startswith("#### T-count:"):
                if start_time:
                    new_lines.append(f"Took: {current_time}\n")
                new_lines.append(line)
                start_time = False
                current_time = 0
            elif line.startswith("Took: "):
                current_time += int(line.split(" ")[1])
                start_time = True

    results = []
    for i in range(0, len(new_lines), 3):
        name = new_lines[i].split('_')[0]
        if name == 'rzk':
            name = 'rz'
        if name == 'crzk':
            name = 'crz'
        k_value = new_lines[i].split('_')[1]
        epsilon = float(new_lines[i].split('_')[2][:-5])
        ms = int(new_lines[i+1][5:-1])
        try:
            t_count = int(new_lines[i+2][13:-6])
        except ValueError:
            t_count = None

        results.append((name + '_' + k_value, epsilon, ms / 1000, t_count))

    dataframe = pd.DataFrame(results, columns=["name", "epsilon", "time", "t_count"])
    return dataframe

### Table 7

In [ ]:
results = []
df = pd.read_csv(source_file, sep='\t', header=None)
df.columns = ['filename', 'time', 'std', 'n_runs', 'successful_runs', 'optimal_runs']
baseline_results = extract_results_baseline_epsilon(os.path.join(gheorghiu_data_path, 'epsilon.txt'))
os.makedirs(os.path.join(output_folder, '65'), exist_ok=True)
for folder in os.listdir(os.path.join(source_output, '65')):
    for subfolder in os.listdir(os.path.join(source_output, '65', folder)):
        output = dict()
        full_folder = os.path.join(source_output, '65', folder, subfolder)
        # search in df for the time
        element = df[df['filename'] == full_folder[6:] + '/'].to_dict('records')[0]
        _, t_count, _, _, best_t_depth_circ, _ = main_analysis(full_folder)
        output['name'] = subfolder
        output['epsilon'] = {'005': 0.05, '10_2': 0.01, '10_3': 0.001}[folder]
        baseline_element = baseline_results[(baseline_results['name'] == subfolder) & (baseline_results['epsilon'] == output['epsilon'])]
        if len(baseline_element) == 0:
            output['baseline_time'] = None
            output['baseline_tcount'] = None
        else:
            output['baseline_time'] = baseline_element['time'].values[0]
            output['baseline_tcount'] = baseline_element['t_count'].values[0]
        if len(t_count) == 0:
            output['time'] = None
            output['tcount'] = None
        else:
            min_t_count = min(t_count)
            number_min_t_count = len(t_count[t_count == min_t_count])
            if element['successful_runs'] >= 100:
                output['time'] = float(element['time'] * element['successful_runs'] / number_min_t_count)
            else:
                output['time'] = float(3600 / number_min_t_count)
            output['tcount'] = min_t_count
            if output['baseline_time'] is not None and output['time'] is not None:
                output['speedup'] = output['baseline_time'] / output['time']

            best_t_depth_circ.circuit.qasm(formatted=True, filename=os.path.join(output_folder, '65', f'{subfolder}_{str(output["epsilon"])}.qasm'))

        results.append(output)
results = pd.DataFrame(results)
results.to_csv(os.path.join(output_folder, '65', 'table7.csv'), index=False)